In [ ]:
import os,time
from dotenv import load_dotenv

from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever


from utils.extractor_gnerico import is_extractive_question, extract_fields_generic


load_dotenv()

print("Bibliotecas importadas correctamente para streaming (Ollama)")

Bibliotecas importadas correctamente para streaming (Ollama)


In [ ]:
# Configuración del modelo con streaming
try:
    llm = ChatOllama(
        base_url=os.getenv("OLLAMA_BASE_URL"),
        model=os.getenv("OLLAMA_MODEL"),
        temperature=0.1,
        num_predict=100,
    )

    print("✓ Modelo configurado con streaming habilitado")
    
    print(f"Modelo: {os.getenv('OLLAMA_MODEL')}")
    print(f"Base URL: {os.getenv('OLLAMA_BASE_URL')}")


    # Warm-up (evita que el primer usuario pague el costo)
    _ = llm.invoke("ping")
    print("✓ Warm-up listo (primer request ya pagado)")

except Exception as e:
    print(f"✗ Error en configuración: {e}")
    raise

docs_txt = TextLoader("data/doctorado_diinf.txt", encoding="utf-8").load()

splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=60)
chunks_txt = splitter.split_documents(docs_txt)

retriever = BM25Retriever.from_documents(chunks_txt)
retriever.k = 4



def is_postgrado_question(q: str) -> bool:
    q = q.lower()
    keywords = [
        "postgrado", "posgrado", "doctorado", "magíster", "magister",
        "arancel", "matrícula", "matricula", "admisión", "admision",
        "postulación", "postulacion", "beca", "requisito", "documento",
        "duración", "duracion", "modalidad", "malla", "líneas", "lineas",
        "acreditación", "acreditacion", "director", "directora", "contacto",
        "cohorte", "semestre"
    ]
    return any(k in q for k in keywords)


def chatbot_streaming():
    print("\n=== CHATBOT CON STREAMING (OLLAMA) ===")
    print("Escribe 'salir' para terminar la conversación\n")

    system_message = (
    "Eres un asistente para ESTUDIANTES de postgrado de la USACH: Universidad de Santiago de Chile.\n"
    "REGLAS ESTRICTAS:\n"
    "1) No inventes datos. Si no tienes información verificable, di: "
    "“No tengo esa información en este momento”.\n"
    "2) No listes programas, fechas, requisitos o valores si no fueron proporcionados "
    "en el contexto o en documentos entregados.\n"
    "3) Si falta un dato, haz UNA sola pregunta de aclaración (ej: programa, cohorte, semestre).\n"
    "4) Responde breve, con pasos accionables.\n"
    "5) Si requiere confirmación oficial, siempre peude sugerir verificar en el sitio oficial https://www.postgradosudesantiago.cl/."
    "6) Usa español natural (tuteo), sin frases raras tipo “por favor dísela”.\n"

)


    while True:
        user_input = input("\n🧑 Tú: ").strip()

        if user_input.lower() in ["salir", "exit", "quit"]:
            print("\n👋 ¡Hasta luego!")
            break

        if not user_input:
            continue

        print("\n🤖 Asistente: ", end="", flush=True)

        try:
            q = user_input.lower()
            
            saludos = ["hola", "buenas", "buenos", "buenas tardes", "buenas noches", "cómo estás", "como estas"]
            is_greeting = any(s in q for s in saludos) and len(q.split()) <= 6
            use_rag = not is_greeting and is_postgrado_question(user_input)



            if use_rag:
                
                BOOST_TERMS = "DIRECTOR/A DIRECTOR CONTACTO TELÉFONOS ADMISIÓN MODALIDAD DURACIÓN MATRÍCULA ARANCEL ACREDITACIÓN"
                query_for_retrieval = f"{user_input} {BOOST_TERMS}"
                docs = retriever.invoke(query_for_retrieval)

                context = "\n\n".join(d.page_content for d in docs)
            
                lc_messages = [
                    SystemMessage(content=system_message),
                    HumanMessage(content=(
                        "Responde usando SOLO el CONTEXTO.\n"
                        "PROHIBIDO: deducir, calcular, estimar, suponer periodicidades o mezclar MATRÍCULA con ARANCEL.\n"
                        "Si el usuario pregunta por MATRÍCULA o ARANCEL y están en el CONTEXTO, responde con el monto EXACTO y su etiqueta.\n"
                        "Si no están en el CONTEXTO, di: “No tengo esa información en este momento” y pide 1 aclaración.\n\n"
                        f"CONTEXTO:\n{context}\n\n"
                        f"PREGUNTA:\n{user_input}"    
                    )),
                ]
            else:
                # ========== CHAT ==========
                lc_messages = [
                   SystemMessage(content=(  
                    system_message + "\n\n"
                    "Eres un chatbot de apoyo para estudiantes de postgrado USACH.\n"
                    "En modo conversación (no-RAG):\n"
                    "- Responde corto y natural, sin prometer 'cualquier consulta académica'.\n"
                    "- Si la pregunta NO es de postgrado USACH, di que no es tu foco y sugiere una fuente apropiada.\n"
                    "Saludo sugerido: “¡Hola! ¿En qué te puedo ayudar sobre postgrados USACH?”"
                )),
                   
                    HumanMessage(content=user_input),
                ]

            # ========== STREAMING ==========
            for chunk in llm.stream(lc_messages):
                content = getattr(chunk, "content", "") or ""
                print(content, end="", flush=True)

                #efecto typing:
                time.sleep(0.01)

            print()  # newline final

        except KeyboardInterrupt:
            print("\n\n⏸️ Interrumpido por el usuario")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")

    print("\n¡Gracias por usar el chatbot!")


if __name__ == "__main__":
    chatbot_streaming()